In [44]:
input = sc.textFile("file:///usr/data/backtothefuture_transcript.txt")

In [45]:
import re
def normalizeWords(text):
    # Strip out the character who says the line (i.e. "Marty:")
    text = re.compile(r'^[^:]+:', re.UNICODE).sub("",text)
    return re.compile(r'\s+', re.UNICODE).split(text.lower())

In [46]:
words = input.flatMap(normalizeWords)

In [47]:
wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [48]:
wordCountsSorted = wordCounts.map(lambda xy: (xy[1], xy[0])).sortByKey(ascending=False)

In [49]:
import numpy as np
import matplotlib.pyplot as plt
top10Words = wordCountsSorted.take(100)
top10Words
#plt.bar(top10Words.keys(), top10Words.values(), width, color='g')

[(1679, ''),
 (323, 'you'),
 (278, 'the'),
 (240, 'i'),
 (178, 'a'),
 (165, 'to'),
 (106, 'that'),
 (103, 'of'),
 (99, 'in'),
 (96, 'your'),
 (87, 'my'),
 (86, 'is'),
 (83, 'this'),
 (81, 'and'),
 (67, "i'm"),
 (65, 'it'),
 (64, 'what'),
 (60, "don't"),
 (58, 'for'),
 (56, 'with'),
 (55, 'gonna'),
 (54, "you're"),
 (48, 'me'),
 (47, 'are'),
 (46, 'be'),
 (44, 'on'),
 (44, "it's"),
 (44, 'do'),
 (44, 'know'),
 (42, 'have'),
 (41, 'just'),
 (41, 'if'),
 (40, 'marty,'),
 (39, 'get'),
 (38, 'about'),
 (37, 'all'),
 (36, 'no'),
 (35, 'out'),
 (35, 'uh,'),
 (34, "that's"),
 (33, 'at'),
 (33, 'doc,'),
 (32, 'it.'),
 (31, 'so'),
 (31, 'well,'),
 (31, 'we'),
 (31, 'no,'),
 (30, 'up'),
 (29, 'was'),
 (29, 'got'),
 (29, 'they'),
 (29, 'but'),
 (29, 'like'),
 (28, 'could'),
 (28, 'one'),
 (27, 'not'),
 (27, 'yeah,'),
 (27, "i'll"),
 (26, 'how'),
 (25, 'oh,'),
 (25, 'hey'),
 (25, 'time'),
 (24, "he's"),
 (24, 'you.'),
 (24, 'look'),
 (24, 'think'),
 (24, 'me.'),
 (23, 'now,'),
 (23, 'him'),
 (23, '

In [63]:
# Get the entire contents of the file, so we can group words by character
contents = sc.wholeTextFiles("file:///usr/data/backtothefuture_transcript.txt")

In [79]:
def normalizeContents(text):
    return re.compile(r'(^[a-z^:]+:)', re.UNICODE|re.MULTILINE).split(text[1].lower())

In [80]:
characterWordCounts = contents.flatMap(normalizeContents)

In [81]:
characterWordCounts.collect()
characterWordCounts.take(100)

['back to the future\n\nproduced by bob gale & neil canton\n\nscreenplay by robert zemeckis & bob gale\n\ndirected by robert zemeckis\n\ntranscribed by eric dienstfrey\n\n',
 'radio:',
 " october is inventory time. so right now, statler toyota is making\nthe best deals of the year on all 1985 model toyotas. you won't find a\nbetter car with a better price with better service anywhere in hill\nvalley...\n\n",
 'television:',
 ' the senate is expected to vote on this today. in other news,\nofficials at the pacific\n\nnuclear research facility have denied the rumor that the case of missing\nplutonium was in fact stolen from their vault two weeks ago. a libyan\nterrorist group had claimed responsibility for the alleged theft,\nhowever, the officials now infer the crepency to a simple clerical\nerror. the\n\nfbi...\n\n',
 'marty:',
 " hey, doc? doc. hello, anybody home? einstein, come here, boy.\nwhat's going on?\n\nwha- aw, god. aw, jesus. whoa, rock and roll. yo\n\n",
 'doc:',
 ' marty, i